In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
!pip install pytorch-lightning
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd

import os
from tqdm import tqdm

import cv2
import torch
import torch.nn as nn

import pytorch_lightning as pl
import timm

In [ ]:
ROOT_DIR = "/content/drive/My Drive/Colab_Notebooks/Zalo2022_LivenessDetection"
GROUP_DIR = f"{ROOT_DIR}/data/Zalo22/LivenessDetection"

os.chdir(ROOT_DIR)

# Cal metrics

In [ ]:
MODEL_NAME = "2D_singleframe_tf_efficientnetv2_s_in21ft1k"
VERSION = "size260_fold0"

print(os.listdir(f"{ROOT_DIR}/checkpoints/{MODEL_NAME}"))

['v1_only_zoom_in', 'size260_fold0', 'face_crop']


In [ ]:
valid_df = pd.read_csv(f"{ROOT_DIR}/checkpoints/{MODEL_NAME}/{VERSION}/valid.csv")

In [ ]:
from utils import equal_error_rate
err, _ = equal_error_rate(valid_df['liveness_score'].tolist(), valid_df['pred_score'].tolist())
print(err)

0.023809523809523808


# Log

In [ ]:
ROOT_DIR = "/content/drive/My Drive/Colab_Notebooks/Zalo2022_LivenessDetection"

version = "baseline_2D_singleframe"

model_dir = f"{ROOT_DIR}/checkpoints/{version}"
event_log_dir = f"{model_dir}/lightning_logs/version_1"

data_dir = f"{ROOT_DIR}/data/public"
test_data_dir = f"{data_dir}/extracted_frames"

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir "./checkpoints/2D_singleframe/lightning_logs/version_0"

# EA

In [ ]:
import zipfile

with zipfile.ZipFile(f"{GROUP_DIR}/data/train.zip", 'r') as zip_ref:
    zip_ref.extractall(f"/content")

# with zipfile.ZipFile(f"{GROUP_DIR}/data/public_test.zip", 'r') as zip_ref:
#     zip_ref.extractall(f"/content")

# with zipfile.ZipFile(f"{GROUP_DIR}/data/public_test_2.zip", 'r') as zip_ref:
#     zip_ref.extractall(f"/content")

## Validation set

In [ ]:
os.listdir(f"{ROOT_DIR}/checkpoints/2D_singleframe_tf_efficientnet_b2_ns/v1_randomframe")

['epoch=13-val_loss=0.077-val_auc=0.998-val_eer=0.024.ckpt',
 'last.ckpt',
 'predict_lb1.csv',
 'predict_lb2.csv',
 'predict_valid.csv']

In [ ]:
model_name = "2D_singleframe_tf_efficientnet_b2_ns"
version = "v1_randomframe"

version_dir = f"{ROOT_DIR}/checkpoints/{model_name}/{version}"
weight_dir = f"{version_dir}/{os.listdir(f'{version_dir}')[0]}"
print(weight_dir)
train_data_dir = f"/content/train"
test_data_dir = f"/content/public"


/content/drive/My Drive/Colab_Notebooks/Zalo2022_LivenessDetection/checkpoints/2D_singleframe_tf_efficientnet_b2_ns/v1_randomframe/epoch=13-val_loss=0.077-val_auc=0.998-val_eer=0.024.ckpt


In [ ]:
os.listdir(f"{version_dir}")

['epoch=13-val_loss=0.077-val_auc=0.998-val_eer=0.024.ckpt',
 'last.ckpt',
 'predict_lb1.csv',
 'predict_lb2.csv',
 'predict_valid.csv']

In [ ]:
# import sys
# sys.path.append(ROOT_DIR)

from models import LivenessModel2D, LivenessModel2DLSTM
from datasets import LivenessDataset

import importlib
cfg = importlib.import_module("configs.config_v3").CFG


In [ ]:
cfg.__dict__

mappingproxy({'__module__': 'configs.config_v3',
              'description': 'Transform config',
              'create_train_transforms': <staticmethod at 0x7f3aa919a910>,
              'create_val_transforms': <staticmethod at 0x7f3aad8f4ed0>,
              '__dict__': <attribute '__dict__' of 'CFG' objects>,
              '__weakref__': <attribute '__weakref__' of 'CFG' objects>,
              '__doc__': None})

In [ ]:
data_df = pd.read_csv(f"/content/drive/My Drive/Colab_Notebooks/Zalo2022_LivenessDetection/data/Zalo22/LivenessDetection/data/meta_data/identified_metadata.csv")
data_df = data_df[data_df.set == "train"]
train_df = data_df[data_df['fold']!=cfg.fold].reset_index(drop=True)
valid_df = data_df[data_df['fold']==cfg.fold].reset_index(drop=True)

train_data_dir = f"/content/train"
cfg.ext = "mp4"
# train_data_dir = f"{ROOT_DIR}/data/train"
# cfg.ext = "jpg"

train_dataset = LivenessDataset(df=train_df, root_dir=train_data_dir, ext=cfg.ext, transforms=cfg.train_transforms, num_frames=cfg.num_frames)
valid_dataset = LivenessDataset(df=valid_df, root_dir=train_data_dir, ext=cfg.ext, transforms=cfg.val_transforms, num_frames=cfg.num_frames)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=False,
                                            num_workers=cfg.num_workers)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=cfg.batch_size, shuffle=False,
                                            num_workers=cfg.num_workers)

In [ ]:
model = LivenessModel2D.load_from_checkpoint(weight_dir, cfg=cfg)
trainer = pl.Trainer()
preds = trainer.predict(model, valid_loader)
preds = torch.cat(preds).detach().cpu().numpy()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

In [ ]:
valid_df["pred_score"] = preds
valid_df.to_csv(f"{version_dir}/predict_valid.csv", index = False)

In [ ]:
valid_df["pred_score"] = preds
valid_df["diff"] = (valid_df["liveness_score"] - valid_df["pred_score"]).abs()
valid_df_sorted = valid_df.sort_values(by=["diff"], ascending=False)
valid_df_sorted.head(10)

,fname,liveness_score,set,knn_queries,individual_id,individual_name,fold,pred_score,diff
24,454.mp4,0.0,train,"[(1193, 0.15370452, nan), (411, 0.16987997, 1....",256,Phương Giang,0.0,0.998569,0.998569
47,870.mp4,1.0,train,"[(103, 0.18741709, 1.0), (108, 0.18899244, 1.0...",464,Ánh Lệ,0.0,0.053332,0.946668
43,811.mp4,0.0,train,"[(1667, 0.073385835, nan), (1862, 0.09363961, ...",431,Linh Duyên,0.0,0.801121,0.801121
78,1483.mp4,0.0,train,"[(827, 0.2934292, 1.0), (1394, 0.31210786, nan...",706,Anh Thi,0.0,0.276167,0.276167
4,97.mp4,0.0,train,"[(1422, 0.39966393, nan), (1368, 0.39966393, n...",66,Minh Danh,0.0,0.233304,0.233304
18,367.mp4,0.0,train,"[(1047, 0.21183121, 0.0), (1274, 0.3044535, na...",210,Duy Thanh,0.0,0.182056,0.182056
41,759.mp4,1.0,train,"[(1359, 0.19729292, nan), (893, 0.21455187, 0....",410,Diệu Linh,0.0,0.903009,0.096991
81,1546.mp4,0.0,train,"[(1289, 5.9604645e-08, nan), (1388, 0.08619523...",731,Thành Phương,0.0,0.092251,0.092251
48,877.mp4,0.0,train,"[(981, 0.020278215, 0.0), (1317, 0.020278215, ...",250,Huệ Thương,0.0,0.085603,0.085603
58,1070.mp4,0.0,train,"[(12, 0.26596797, 0.0), (954, 0.2704457, 0.0),...",545,Thụy Ðào,0.0,0.070381,0.070381


In [ ]:
train_df

,fname,liveness_score,set,knn_queries,individual_id,individual_name,fold
0,1.mp4,0.0,train,"[(108, 0.24212176, 1.0), (11, 0.25579798, 1.0)...",0,Hồng Oanh,9.0
1,2.mp4,1.0,train,"[(485, 0.023995936, 1.0), (593, 0.29561955, 0....",1,Vĩnh Hưng,4.0
2,3.mp4,1.0,train,"[(944, 0.22763091, 1.0), (987, 0.2309981, 1.0)...",2,Hoàn Vũ,1.0
3,5.mp4,0.0,train,"[(1500, 0.026418447, nan), (566, 0.09758127, 0...",3,Vĩnh Luân,5.0
4,7.mp4,1.0,train,"[(731, 0.22429407, 1.0), (189, 0.24795604, 1.0...",4,Hà Hải,6.0
...,...,...,...,...,...,...,...
900,1877.mp4,1.0,train,"[(177, 0.10502881, 1.0), (352, 0.12476659, 0.0...",164,Vân Du,8.0
901,1879.mp4,1.0,train,"[(1724, 0.25729442, nan), (203, 0.2586382, 0.0...",857,Tiểu My,8.0
902,1880.mp4,1.0,train,"[(1822, 0.13246047, nan), (868, 0.13650721, 1....",858,Tuấn Hùng,9.0
903,1881.mp4,0.0,train,"[(1106, 0.13971096, 1.0), (1393, 0.20642453, n...",859,Hạ Phương,1.0


In [ ]:
valid_df["pred_score"] = preds
valid_df["diff"] = (valid_df["liveness_score"] - valid_df["pred_score"]).abs()
valid_df_sorted = valid_df.sort_values(by=["diff"], ascending=False)
valid_df_sorted.head(10)

,fname,liveness_score,set,knn_queries,individual_id,individual_name,fold,pred_score,diff
43,811.mp4,0.0,train,"[(1667, 0.073385835, nan), (1862, 0.09363961, ...",431,Linh Duyên,0.0,0.997928,0.997928
47,870.mp4,1.0,train,"[(103, 0.18741709, 1.0), (108, 0.18899244, 1.0...",464,Ánh Lệ,0.0,0.131635,0.868365
24,454.mp4,0.0,train,"[(1193, 0.15370452, nan), (411, 0.16987997, 1....",256,Phương Giang,0.0,0.851169,0.851169
58,1070.mp4,0.0,train,"[(12, 0.26596797, 0.0), (954, 0.2704457, 0.0),...",545,Thụy Ðào,0.0,0.313254,0.313254
4,97.mp4,0.0,train,"[(1422, 0.39966393, nan), (1368, 0.39966393, n...",66,Minh Danh,0.0,0.185065,0.185065
51,945.mp4,1.0,train,"[(825, 0.17243189, 0.0), (1953, 0.18274349, na...",494,Hồng Việt,0.0,0.870697,0.129303
11,222.mp4,1.0,train,"[(1439, 0.042743027, nan), (1686, 0.056779385,...",135,Thế An,0.0,0.938911,0.061089
81,1546.mp4,0.0,train,"[(1289, 5.9604645e-08, nan), (1388, 0.08619523...",731,Thành Phương,0.0,0.059711,0.059711
32,568.mp4,0.0,train,"[(1319, 0.051050365, nan), (1521, 0.08502811, ...",311,Tú Sương,0.0,0.052248,0.052248
71,1322.mp4,1.0,train,"[(930, 0.14784187, 0.0), (43, 0.17430395, 0.0)...",648,Vạn Hạnh,0.0,0.960054,0.039946


## Test set

In [ ]:
version = "baseline_2D"
weight_name = "epoch=22-val_loss=0.10-val_f1=0.98.ckpt"
event_name = "events.out.tfevents.1667961578.a270725d95ab.1121.0"

model_dir = f"{ROOT_DIR}/checkpoints/{version}"
weight_dir = f"{model_dir}/lightning_logs/version_1/checkpoints/{weight_name}"
event_log_dir = f"{model_dir}/lightning_logs"

train_data_dir = f"/content/train"
test_data_dir = f"/content/public_test_2"
type_data_dir = "extracted_frames"

In [ ]:
# import sys
# sys.path.append(ROOT_DIR)

from models import LivenessModel2D
from datasets import LivenessDataset

import importlib
cfg = importlib.import_module("configs.config0").CFG

In [ ]:
fnames = sorted(os.listdir(f"{test_data_dir}/videos"), key=lambda x: int(x.split(".")[0]))
test_df = pd.DataFrame({"fname": fnames})
test_dataset = LivenessDataset(df=test_df, root_dir=f"{test_data_dir}", transforms=cfg.val_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False,
                                            num_workers=cfg.num_workers)

In [ ]:
predict_df = pd.read_csv(f"{model_dir}/predict.csv")
predict_df

,Unnamed: 0,fname,liveness_score
0,0,0.mp4,0.999999
1,1,4.mp4,0.984693
2,2,15.mp4,0.524029
3,3,17.mp4,0.000831
4,4,21.mp4,0.000086
...,...,...,...
345,345,1864.mp4,0.998475
346,346,1867.mp4,0.999351
347,347,1878.mp4,0.000245
348,348,1883.mp4,0.998183


In [ ]:
import matplotlib.pyplot as plt

predict_df = predict_df.sort_values(by=["liveness_score"], ascending=False)
for col in predict_df.iterrows():
    fname = col[1].fname.split(".")[0]
    score = col[1].liveness_score 
    
    if score < 0.4 or score > 0.6:
        continue

    plt.figure(figsize=(12,6))
    plt.title(f"fname: {fname} | liveness_score: {score}")

    im = plt.imread(f"{test_data_dir}/extracted_frames/{fname}/0.jpg")
    plt.imshow(im)
    plt.show()

# Inference

In [ ]:
import zipfile

with zipfile.ZipFile(f"{GROUP_DIR}/data/train.zip", 'r') as zip_ref:
    zip_ref.extractall(f"/content")

with zipfile.ZipFile(f"{GROUP_DIR}/data/public_test.zip", 'r') as zip_ref:
    zip_ref.extractall(f"/content")

with zipfile.ZipFile(f"{GROUP_DIR}/data/public_test_2.zip", 'r') as zip_ref:
    zip_ref.extractall(f"/content")

In [ ]:
os.listdir(f"{ROOT_DIR}/checkpoints/2D_singleframe_tf_efficientnetv2_s_in21ft1k")

['face_crop', 'size384_cropface', 'v1_only_zoom_in', 'size260_fold0']

In [ ]:
model_name = "2D_singleframe_tf_efficientnetv2_s_in21ft1k"
version = "size384_cropface"

version_dir = f"{ROOT_DIR}/checkpoints/{model_name}/{version}"
weight_dir = f"{version_dir}/{os.listdir(f'{version_dir}')[-2]}"
print(weight_dir)

train_data_dir = f"/content/train"
test_data_dir = f"/content/public"
test_data_dir2 = f"/content/public_test_2"


/content/drive/My Drive/Colab_Notebooks/Zalo2022_LivenessDetection/checkpoints/2D_singleframe_tf_efficientnetv2_s_in21ft1k/size384_cropface/epoch=29-val_loss=0.065-val_auc=0.999-val_eer=0.024.ckpt


In [ ]:
os.listdir(f"{version_dir}")

['opt.json',
 'epoch=6-val_loss=0.074-val_auc=0.997-val_eer=0.024.ckpt',
 'epoch=9-val_loss=0.072-val_auc=0.998-val_eer=0.024.ckpt',
 'epoch=10-val_loss=0.075-val_auc=0.997-val_eer=0.048.ckpt',
 'epoch=17-val_loss=0.070-val_auc=0.998-val_eer=0.024.ckpt',
 'epoch=24-val_loss=0.073-val_auc=0.996-val_eer=0.024.ckpt',
 'epoch=29-val_loss=0.065-val_auc=0.999-val_eer=0.024.ckpt',
 'last.ckpt']

In [ ]:
# import sys
# sys.path.append(ROOT_DIR)

from models import LivenessModel2D, LivenessModel2DLSTM
from datasets import LivenessDataset

In [ ]:
import json

class Dict2Class(object):
    def __init__(self, my_dict):
          
        for key in my_dict:
            setattr(self, key, my_dict[key])

cfg = json.load(open(f"{version_dir}/opt.json"))
cfg = Dict2Class(cfg)

import importlib
transform_cfg = importlib.import_module(f'configs.{cfg.transform_config}').CFG

In [ ]:
# valid

cfg.num_frames = 1
cfg.frame_step = -1

data_df = pd.read_csv(f"/content/drive/My Drive/Colab_Notebooks/Zalo2022_LivenessDetection/data/Zalo22/LivenessDetection/data/meta_data/identified_metadata.csv")
data_df = data_df[data_df.set == "train"]
train_df = data_df[data_df['fold']!=cfg.fold].reset_index(drop=True)
valid_df = data_df[data_df['fold']==cfg.fold].reset_index(drop=True)

valid_dataset = LivenessDataset(df=valid_df, root_dir=train_data_dir, ext=cfg.ext, transforms=transform_cfg.create_val_transforms(cfg.height, cfg.width), num_frames=cfg.num_frames)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=cfg.batch_size, shuffle=False,
                                            num_workers=cfg.num_workers)

model = LivenessModel2D.load_from_checkpoint(weight_dir, cfg=cfg)
trainer = pl.Trainer(accelerator='cpu')
preds = trainer.predict(model, valid_loader)

preds = torch.cat(preds).detach().cpu().numpy()
valid_df["pred_score"] = preds
valid_df.to_csv(f"{version_dir}/valid.csv", index = False)

Use 1 frame | Load image from mp4 | Face crop False


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

In [ ]:
# LB2

cfg.num_frames = 5
cfg.frame_step = -1

fnames = sorted(os.listdir(f"{test_data_dir2}/videos"), key=lambda x: int(x.split(".")[0]))
test_df = pd.DataFrame({"fname": fnames})
test_dataset = LivenessDataset(df=test_df, root_dir=test_data_dir2, transforms=transform_cfg.create_val_transforms(cfg.height, cfg.width), num_frames=cfg.num_frames, ext=cfg.ext, frame_step=cfg.frame_step)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False,
                                            num_workers=cfg.num_workers)

model = LivenessModel2D.load_from_checkpoint(weight_dir, cfg=cfg)
trainer = pl.Trainer(accelerator='cpu')
preds = trainer.predict(model, test_loader)

preds = torch.cat(preds).detach().cpu().numpy()
test_df["liveness_score"] = preds
test_df.to_csv(f"{version_dir}/predict_lb2_5frames.csv", index = False)

Use 5 frame | Load image from mp4 | Face crop False


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

In [ ]:
# LB1
cfg.num_frames = 5
cfg.frame_step = -1

fnames = sorted(os.listdir(f"{test_data_dir}/videos"), key=lambda x: int(x.split(".")[0]))
test_df = pd.DataFrame({"fname": fnames})
test_dataset = LivenessDataset(df=test_df, root_dir=test_data_dir, transforms=transform_cfg.create_val_transforms(cfg.height, cfg.width), num_frames=cfg.num_frames, ext=cfg.ext)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False,
                                            num_workers=cfg.num_workers)

model = LivenessModel2D.load_from_checkpoint(weight_dir, cfg=cfg)
trainer = pl.Trainer(accelerator='cpu')
preds = trainer.predict(model, test_loader)

preds = torch.cat(preds).detach().cpu().numpy()
test_df["liveness_score"] = preds
test_df.to_csv(f"{version_dir}/predict_lb1_5frame.csv", index = False)

Use 5 frame | Load image from mp4 | Face crop False


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]